<a href="https://colab.research.google.com/github/AishaLichtner/projects-2019/blob/master/Important_features.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from __future__ import absolute_import, division, print_function, unicode_literals
import os
import tensorflow.compat.v1 as tf
from tensorflow import keras

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D,Conv1D,Conv3D, MaxPool2D , Flatten
from tensorflow.keras.preprocessing.image import ImageDataGenerator


from tensorflow.keras.wrappers.scikit_learn import KerasRegressor
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler,RobustScaler
from sklearn.pipeline import Pipeline

from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import Ridge



from tensorflow.keras import optimizers as opt
import tensorflow.keras.backend as K
import pathlib

import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns

import datetime
import statistics


#get last sunday of the month
import sys
import calendar

from tensorflow.keras import layers

print(tf.__version__)

import numpy as np     # matrix structures, linear algebra functions etc
from sklearn.linear_model import LinearRegression   # linear regression model
from sklearn.metrics import mean_squared_error, mean_absolute_error   # accuracy metrics


from collections import Counter

import io

1.15.0
1.15.0


In [0]:
from google.colab import files
uploaded = files.upload()

In [0]:
uploaded

{}

In [0]:
from google.colab import drive

drive.mount('/content/drive')
path = "/content/drive/My Drive/data/blocks.csv"
path2 ="/content/drive/My Drive/data/flightpairs.csv"
data = pd.read_csv(path,sep = ';', encoding="utf-8-sig")
flightpairs= pd.read_csv(path2,sep = ';', encoding="utf-8-sig")
# Dataset is now stored in a Pandas Dataframe

In [0]:

flightpairs=pd.DataFrame(flightpairs)
data

In [0]:
#exclude flights that are operated by other companies
data = data.loc[data["AOC"] != "WET"]


In [0]:
flightpairs.tail()
data.tail()
print(flightpairs.values)
#delete rows with invalid entries (nan-values)
data = data.dropna()
print(len(flightpairs))
flightpairs=flightpairs.dropna()
print(len(flightpairs))

Filter by Season

In [0]:
#exclude date colum of flightBlock so we can split it in more information
date= data.pop("Std UTC")
date

In [0]:

def lastSunday(year,month):
    if len(sys.argv) > 1:
        try:
            year = int(sys.argv[-1])
        except ValueError:
            pass
 
   
    last_sunday = max(week[-1] for week in calendar.monthcalendar(year, month))
    #print('{}-{}-{:2}'.format(year, calendar.month_abbr[month], last_sunday))
    return last_sunday


In [0]:
#returns 1 for winterseason and 0 for summer season
def getSeason(d):

    if (d.month<3 or d.month > 10):
        return 1;
    elif (d.month == 3 and d.day < lastSunday(d.year, d.month)):
        return 1;
    elif (d.month == 10 and d.day >= lastSunday(d.year, d.month)):
        return 1;
    else:
        return 0;
    

In [0]:
import random
#create some more categories like time of the day, year, weekday and season
#date is the column of flightBlock 
dateForm=[datetime.datetime.strptime(s,'%d.%m.%Y %H:%M') for s in date]
#add some random small number to the time
times= [x.hour  for x in dateForm]
weekdays =[x.weekday()+1 for x in dateForm]
months= [x.month for x in dateForm]
summer=[(getSeason(x)==0)*1  for x in dateForm]

#timesUni= [x.hour + random.uniform(0,0.5) for x in dateForm]
#weekdaysUni =[x.weekday()+1+ random.uniform(0,0.5) for x in dateForm]
#monthsUni= [x.month + random.uniform(0,0.5) for x in dateForm]


In [0]:
dataUni=data.copy()

data['Time'] = times
data["Weekday"] = weekdays
data["Summer"] = summer
data["Month"]=months
#data["Summer"]


#dataUni['Time'] = timesUni
#dataUni["Weekday"] = weekdaysUni
#dataUni["Summer"] = summer
#dataUni["Month"]=monthsUni

In [0]:
data["Delay"]=(data["ActualBlock"]-data["BlkSched"])

In [0]:
data.tail()

In [0]:
def createCityPairSummer(str_citypair):
    cityPair=pd.DataFrame
    cityPair= data.loc[((data['DP Iatacode']+ data['AR Iatacode'] == str_citypair) & data["Summer"])]
    return cityPair.dropna()

In [0]:
def createCityPairWinter(str_citypair):
    cityP=pd.DataFrame
    cityP= data.loc[((data['DP Iatacode']+ data['AR Iatacode'] == str_citypair) & (data["Summer"]==0))]
    return cityP.dropna()

In [0]:
createCityPairSummer("BOSFCO")

In [0]:
createCityPairWinter("BOSFCO")

In [0]:
listOfPairs= []
print(type(flightpairs.values[0]))
for i in flightpairs.values:
        if(i[0] != "NaN"):
            summer=createCityPairSummer(i[0])
            summer=summer.dropna()
            winter=createCityPairWinter(i[0])
            winter=winter.dropna()
#Finde den boesewicht 
#           if(i[0] == "BOSFCO"):
 #               print("hab die Arschgeige")
        
            if(len(summer) > 10): 
                listOfPairs.append(summer)
            if(len(winter) > 10):
                listOfPairs.append(winter)
           


In [0]:
len(listOfPairs)

In [0]:
def late(liste):
    lateLimit= 10
    latePerc=0.3
    a=sum((abs(liste["ActualBlock"]-liste["BlkSched"])> lateLimit)*1)
    print (a/len(liste))
    return (a/len(liste)>latePerc)

In [0]:
late(listOfPairs[0])
l= len(listOfPairs[0])
i=0
print(type(listOfPairs[0]))
for x in listOfPairs[0]["Delay"]:
        if(abs(x) > 10):
            i=i+1
print(l)            
print(i/l)

In [0]:
#filer flights that are operated less than 50 times
bound= 50
listOfPairs = list(filter(lambda i: len(i)> 50 , listOfPairs))
#filter flights that are less than 30% of time delayed
listOfPairs = list(filter(late, listOfPairs ))


# Find and delete outliers

In [0]:
liste = []
for flight in listOfPairs:


    oldSize=flight.shape[0]
    Q1 = flight["ActualBlock"].quantile(0.25)
    Q3 = flight["ActualBlock"].quantile(0.75)
    IQR = Q3 - Q1
    print(IQR)

    #outliars beeing filtered
    flight = flight[~((flight["ActualBlock"] < (Q1 - 1.5 * IQR)) |(flight["ActualBlock"] > (Q3 + 1.5 * IQR)))]
    print("percentage of non-outlier flights:")
    print(flight.shape[0]/oldSize )
    liste.append(flight)
    
listOfFlights=liste

In [0]:
pd.DataFrame(listOfFlights).to_excel("listOfFlights.xlsx")

In [0]:
def delayToTarget(delay):
    target=[ "late" if x > 10  else( "early"  if x < -10  else "on time")  for x in delay]
    target= pd.DataFrame(target)
    return target

#print(delayToTarget(listOfPairs[0]["Delay"]))
print(listOfPairs[12]["Delay"])

import scipy.linalg as la
def getEigenValVec(flight):
        # Separating out the features
    x = flight.loc[:, ["Time", "Weekday", "Month"]].values
    #zentrieren
    x = StandardScaler().fit_transform(x)
    print(x.shape)
    C= np.cov(np.transpose(x))
    results = la.eig(C)
    return results
  

In [0]:
#plot it in 3D


    
from mpl_toolkits.mplot3d import Axes3D

for flight in listOfPairs:
    print(flight["Delay"])
    fig = plt.figure(1, figsize=(7, 12))
    ax = Axes3D(fig, elev=-150, azim=-110)
    ax.set_title(flight.iloc[0]["DP Iatacode"]+flight.iloc[0]["AR Iatacode"]+" in Season (1= summer,0=winter) "+ str(flight.iloc[0]["Summer"] ))
    ax.set_xlabel("Time")
    ax.w_xaxis.set_ticklabels([])
    ax.set_ylabel("Weekday")
    ax.w_yaxis.set_ticklabels([])
    ax.set_zlabel("Month")
    ax.w_zaxis.set_ticklabels([])
    
    targets = ['late', 'early', 'on time']
    colors = ['r', 'g', 'b']
 
    for target, color in zip(targets,colors):
        indicesToKeep = delayToTarget(flight['Delay']) == target
        indicesToKeep =indicesToKeep.values
        #print(flight.loc[indicesToKeep])
        ax.scatter(flight[indicesToKeep]['Time'], flight[indicesToKeep]['Weekday'], flight[indicesToKeep]['Month'],
                   edgecolor='k', s=40, c=color)

    ax.legend(targets)


    plt.show()

In [0]:
#3d pca with blocktimes


#for flight in listOfPairs:
  #  d = doPCA(flight)
    #eigenvectors=getEigenvectors(flight)
    
    
  #  fig = plt.figure(1, figsize=(7, 12))
 #   ax = Axes3D(fig, elev=-150, azim=-110)
   # ax.set_title(flight.iloc[0]["DP Iatacode"]+flight.iloc[0]["AR Iatacode"])
   # ax.set_xlabel("PC1")
    #ax.w_xaxis.set_ticklabels([])
 #   ax.set_ylabel("PC2")
   # ax.w_yaxis.set_ticklabels([])
   # ax.set_zlabel("Actual Block")
  #  ax.w_zaxis.set_ticklabels([])
    
    
    
  #  targets = ['late', 'early']
   # colors = ['r', 'g', 'b']
 
  #{ for target, color in zip(targets,colors):
    #    indicesToKeep = delayToTarget(flight['Delay']) == target
      #  indicesToKeep =indicesToKeep.values
     #   ax.scatter(d[indicesToKeep]["principal component 1"], d[indicesToKeep]["principal component 2"],flight[indicesToKeep]["ActualBlock"],
              #     edgecolor='k', s=40, c=color)

  #  ax.legend(targets)   
  #  plt.show()
    
        
   # targets = ['late', 'early']
  #  colors = ['r', 'g', 'b']
 #2d pca
   # for target, color in zip(targets,colors):
    #    indicesToKeep = delayToTarget(flight['Delay']) == target
    #    indicesToKeep =indicesToKeep.values
    #    plt.scatter(d[indicesToKeep]["principal component 1"], d[indicesToKeep]["principal component 2"],
                   #edgecolor='k', s=40, c=color)
        
      #  plt.title(flight.iloc[0]["DP Iatacode"]+flight.iloc[0]["AR Iatacode"])
     #   plt.xlabel("PC1")
      #  plt.ylabel("PC2")
  #  plt.legend(targets)   
  # plt.show() 

In [0]:
#try different idea: Tree Regressor, maybe better because there is continous targets

from sklearn.ensemble import ExtraTreesRegressor
from sklearn import preprocessing
from sklearn import utils

flightMonth=[]
feat=pd.DataFrame(columns=["Flight","Season","Month","Weekday","Time"])
for flight in listOfPairs:

    #last cleaning troubleasshole 251553 
    flight=flight.dropna()
    # Separating out the features
    x = flight.loc[:, ["Time", "Weekday", "Month"]]
    # Separating out the target
    y = flight.loc[:,['Delay']]
    
    
    
    
    
    flightName = flight.iloc[0]["DP Iatacode"]+flight.iloc[0]["AR Iatacode"]
    seas = flight.iloc[0]["Summer"]

    

    model = ExtraTreesRegressor()
    model.fit(x,y)
 
    #write in dataframe so we can produce excel file
    
        

    
    #plot graph of feature importances for better visualization
    feat_importances = pd.Series(model.feature_importances_, index=x.columns)
    feat_importances.nlargest(3).plot(kind='barh')
    print(feat_importances)
    
    # Append rows in Empty Dataframe by adding dictionaries
    feat = feat.append({'Flight': flightName, 'Season': seas, 'Month':feat_importances["Month"],"Weekday": feat_importances["Weekday"] ,"Time": feat_importances["Time"] }, ignore_index=True)

    
    
    plt.title(flightName +" in Season (1= summer,0=winter) "+ str(seas ))
    plt.show()
    print("this is the importance")
    print(model.feature_importances_)#use inbuilt class feature_importances of tree based classifiers
    
    if((feat_importances["Month"] == feat_importances.max()) & (flightName not in flightMonth) &(feat_importances.max() > 0.7) ):
        flightMonth.append(flightName)
    


In [0]:
feat.to_excel("importantFeatures.xlsx")
#flightMonth.to_csv("monatFluege.csv")


In [0]:
len(flightMonth)

In [0]:
#sucht alle fluege raus flughafen paar str_citypair sind und loescht direkt outliers
def createCityPairAll(str_citypair):
    cityPair=pd.DataFrame
    cityPair= data.loc[(data['DP Iatacode']+ data['AR Iatacode'] == str_citypair)]
    
    oldSize=cityPair.shape[0]
    Q1 = cityPair["ActualBlock"].quantile(0.25)
    Q3 = cityPair["ActualBlock"].quantile(0.75)
    IQR = Q3 - Q1


    #outliars beeing filtered
    cityPair = cityPair[~((cityPair["ActualBlock"] < (Q1 - 1.5 * IQR)) |(cityPair["ActualBlock"] > (Q3 + 1.5 * IQR)))]


    return cityPair.dropna()

# make some nice plots

In [0]:
import matplotlib.backends.backend_pdf
pdf = matplotlib.backends.backend_pdf.PdfPages("Allplots.pdf")

for flight in flightMonth:
        
        alle=createCityPairAll(flight)
        fig, axs = plt.subplots(12, 7, figsize=(30,30),sharex='all', sharey='all')
        for i in range(1,12):
            for k in range(1,7):
                blockT = alle[(alle["Month"] == i) & (alle["Weekday"] == k)]
                if(not blockT.empty):
                    
                   # plt.set_xlim([blockT["ActualBlock"].min(),blockT["ActualBlock"].max()])
                    axs[i-1, k-1].hist(blockT["ActualBlock"])
                    axs[i-1, k-1].set_title('month: '+ str(i) +"weekday: "+str(k),fontsize='small')
               
        fig.suptitle(flight)
        plt.subplots_adjust(hspace=0.4, wspace=0.2)
        plt.show()
        pdf.savefig( fig )
    



pdf.close()

>>> import matplotlib.pyplot as plt
>>> rng = np.random.RandomState(10)  # deterministic random data
>>> a = np.hstack((rng.normal(size=1000),
...                rng.normal(loc=5, scale=2, size=1000)))
>>> _ = plt.hist(a, bins='auto')  # arguments are passed to np.histogram
>>> plt.title("Histogram with 'auto' bins")
Text(0.5, 1.0, "Histogram with 'auto' bins")
>>> plt.show()

        plt.tight_layout()
f.savefig("foo.pdf", bbox_inches='tight')